In [16]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import euclidean
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sqlalchemy import create_engine
from dotenv import load_dotenv
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import pymysql
import os

In [17]:
import sys
import os
sys.path.append(os.path.abspath('../scripts'))

In [18]:
from load_data import load_data_from_postgres, load_data_using_sqlalchemy
from sql_queries import execute_telecom_queries

In [19]:
# Define your SQL query
query = "SELECT * FROM xdr_data;"  # Replace with your actual table name

# Load data from PostgreSQL
data = load_data_from_postgres(query)

# Display the first few rows of the dataframe
if data is not None:
    print("Successfully loaded the data")
else:
    print("Failed to load data.")


d:\Projects\Telcom\scripts\load_data.py:38: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, connection)


Successfully loaded the data


In [20]:
data

,Bearer Id,Start,Start ms,End,End ms,Dur. (ms),IMSI,MSISDN/Number,IMEI,Last Location Name,...,Youtube DL (Bytes),Youtube UL (Bytes),Netflix DL (Bytes),Netflix UL (Bytes),Gaming DL (Bytes),Gaming UL (Bytes),Other DL (Bytes),Other UL (Bytes),Total UL (Bytes),Total DL (Bytes)
0,1.311448e+19,4/4/2019 12:01,770.0,4/25/2019 14:35,662.0,1823652.0,2.082014e+14,3.366496e+10,3.552121e+13,9.16456699548519E+015,...,1.585461e+07,2.501332e+06,8.198936e+06,9.656251e+06,2.780823e+08,1.434415e+07,1.717444e+08,8.814393e+06,36749741.0,308879636.0
1,1.311448e+19,4/9/2019 13:04,235.0,4/25/2019 8:15,606.0,1365104.0,2.082019e+14,3.368185e+10,3.579401e+13,L77566A,...,2.024740e+07,1.911173e+07,1.833841e+07,1.722713e+07,6.087501e+08,1.170709e+06,5.269042e+08,1.505514e+07,53800391.0,653384965.0
2,1.311448e+19,4/9/2019 17:42,1.0,4/25/2019 11:58,652.0,1361762.0,2.082003e+14,3.376063e+10,3.528151e+13,D42335A,...,1.972566e+07,1.469958e+07,1.758779e+07,6.163408e+06,2.295846e+08,3.956300e+05,4.106926e+08,4.215763e+06,27883638.0,279807335.0
3,1.311448e+19,4/10/2019 0:31,486.0,4/25/2019 7:36,171.0,1321509.0,2.082014e+14,3.375034e+10,3.535661e+13,T21824A,...,2.138812e+07,1.514664e+07,1.399465e+07,1.097942e+06,7.995382e+08,1.084972e+07,7.490399e+08,1.279728e+07,43324218.0,846028530.0
4,1.311448e+19,4/12/2019 20:10,565.0,4/25/2019 10:40,954.0,1089009.0,2.082014e+14,3.369980e+10,3.540701e+13,D88865A,...,1.525938e+07,1.896287e+07,1.712458e+07,4.152180e+05,5.277072e+08,3.529801e+06,5.507095e+08,1.391032e+07,38542814.0,569138589.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
149996,7.277826e+18,4/29/2019 7:28,451.0,4/30/2019 6:02,214.0,81230.0,2.082022e+14,3.365069e+10,3.548311e+13,D20434A,...,1.619167e+07,1.176343e+07,1.788370e+07,1.967816e+07,5.266097e+08,9.197207e+06,3.264510e+06,1.348742e+07,57628851.0,574175259.0
149997,7.349883e+18,4/29/2019 7:28,483.0,4/30/2019 10:41,187.0,97970.0,2.082019e+14,3.366345e+10,3.566051e+13,D10223C,...,1.387723e+07,8.288284e+06,1.935015e+07,2.129315e+07,6.268931e+08,4.735033e+06,7.121804e+08,2.457758e+06,39135081.0,666648844.0
149998,1.311448e+19,4/29/2019 7:28,283.0,4/30/2019 10:46,810.0,98249.0,2.082017e+14,3.362189e+10,3.572121e+13,T51102A,...,2.266051e+07,1.855903e+06,9.963942e+06,5.065760e+06,5.535395e+08,1.339432e+07,1.211009e+08,1.131473e+07,34912224.0,592786405.0
149999,1.311448e+19,4/29/2019 7:28,696.0,4/30/2019 10:40,327.0,97910.0,2.082021e+14,3.361962e+10,8.618620e+13,L88342B,...,8.817106e+06,8.305402e+06,3.322253e+06,1.317259e+07,3.525370e+08,2.529475e+06,8.147131e+08,1.406930e+06,29626096.0,371895920.0


In [21]:
# Step 2: Handle Missing Values and Treat Outliers
experience_numeric_cols = [
    'TCP DL Retrans. Vol (Bytes)',
    'TCP UL Retrans. Vol (Bytes)',
    'Avg RTT DL (ms)',
    'Avg RTT UL (ms)',
    'Avg Bearer TP DL (kbps)',
    'Avg Bearer TP UL (kbps)'
]
engagement_numeric_cols = ['Dur. (ms)', 'Total DL (Bytes)', 'Total UL (Bytes)']

for col in experience_numeric_cols + engagement_numeric_cols:
    if col in data.columns:
        mean_val = data[col].mean()
        data[col].fillna(mean_val, inplace=True)
        q1 = data[col].quantile(0.25)
        q3 = data[col].quantile(0.75)
        iqr = q3 - q1
        lower = q1 - 1.5 * iqr
        upper = q3 + 1.5 * iqr
        mask = (data[col] < lower) | (data[col] > upper)
        data.loc[mask, col] = mean_val

if 'Handset Type' in data.columns:
    handset_mode = data['Handset Type'].mode(dropna=True)
    if not handset_mode.empty:
        data['Handset Type'].fillna(handset_mode.iloc[0], inplace=True)

load_dotenv()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_13064\2113335459.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data[col].fillna(mean_val, inplace=True)
C:\Users\Administrator\AppData\Local\Temp\ipykernel_13064\2113335459.py:15: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy

True

In [22]:
# Step 3: Aggregate Engagement Metrics per User (Task 2 logic)
engagement_data = data.groupby('MSISDN/Number').agg({
    'Bearer Id': 'count',
    'Dur. (ms)': 'sum',
    'Total DL (Bytes)': 'sum',
    'Total UL (Bytes)': 'sum'
}).reset_index()

engagement_data.rename(columns={
    'MSISDN/Number': 'MSISDN',
    'Bearer Id': 'Session Frequency',
    'Dur. (ms)': 'Total Session Duration',
    'Total DL (Bytes)': 'Total DL (Bytes)',
    'Total UL (Bytes)': 'Total UL (Bytes)'
}, inplace=True)

engagement_data['Total Traffic (Bytes)'] = engagement_data['Total DL (Bytes)'] + engagement_data['Total UL (Bytes)']

engagement_features = engagement_data[['Session Frequency', 'Total Session Duration', 'Total Traffic (Bytes)']]

engagement_scaler = MinMaxScaler()
engagement_scaled = engagement_scaler.fit_transform(engagement_features)

engagement_kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
engagement_clusters = engagement_kmeans.fit_predict(engagement_scaled)
engagement_data['Engagement Cluster'] = engagement_clusters

engagement_centers = engagement_kmeans.cluster_centers_
engagement_center_df = pd.DataFrame(engagement_centers, columns=engagement_features.columns)
engagement_center_df

,Session Frequency,Total Session Duration,Total Traffic (Bytes)
0,0.233128,0.211853,0.245782
1,0.057704,0.041920,0.052807
2,0.119474,0.092408,0.122571


In [23]:
# Step 4: Aggregate Experience Metrics per User (Task 3 logic)
experience_data = data.groupby('MSISDN/Number').agg({
    'TCP DL Retrans. Vol (Bytes)': 'mean',
    'TCP UL Retrans. Vol (Bytes)': 'mean',
    'Avg RTT DL (ms)': 'mean',
    'Avg RTT UL (ms)': 'mean',
    'Avg Bearer TP DL (kbps)': 'mean',
    'Avg Bearer TP UL (kbps)': 'mean',
    'Handset Type': lambda x: x.mode().iloc[0] if not x.mode().empty else x.iloc[0]
}).reset_index()

experience_data.rename(columns={'MSISDN/Number': 'MSISDN'}, inplace=True)

experience_data['Average TCP Retransmission'] = (
    experience_data['TCP DL Retrans. Vol (Bytes)'] + experience_data['TCP UL Retrans. Vol (Bytes)']
) / 2
experience_data['Average RTT'] = (
    experience_data['Avg RTT DL (ms)'] + experience_data['Avg RTT UL (ms)']
) / 2
experience_data['Average Throughput'] = (
    experience_data['Avg Bearer TP DL (kbps)'] + experience_data['Avg Bearer TP UL (kbps)']
) / 2

experience_features = experience_data[['Average TCP Retransmission', 'Average RTT', 'Average Throughput']]
experience_scaler = MinMaxScaler()
experience_scaled = experience_scaler.fit_transform(experience_features)

experience_kmeans = KMeans(n_clusters=3, random_state=42, n_init=10)
experience_clusters = experience_kmeans.fit_predict(experience_scaled)
experience_data['Experience Cluster'] = experience_clusters

experience_centers = experience_kmeans.cluster_centers_
experience_center_df = pd.DataFrame(
    experience_scaler.inverse_transform(experience_centers),
    columns=experience_features.columns
)
experience_center_df

,Average TCP Retransmission,Average RTT,Average Throughput
0,9.519061e+06,21.798785,789.261461
1,2.488799e+06,40.981727,11809.732709
2,8.660230e+06,62.758078,1046.318422


In [24]:
# Reuse normalized experience metrics for downstream steps
experience_metrics = experience_scaled.copy()
engagement_metrics = engagement_scaled.copy()

In [25]:
# Step 5: Compute Engagement and Experience Scores (Euclidean distances)
least_engaged_cluster = np.argmin(engagement_centers.sum(axis=1))
engagement_scores = np.linalg.norm(
    engagement_scaled - engagement_centers[least_engaged_cluster], axis=1
)

experience_penalty = (
    experience_centers[:, 0]  # high retransmission is bad
    + experience_centers[:, 1]  # high RTT is bad
    + (1 - experience_centers[:, 2])  # low throughput is bad
)
worst_experience_cluster = np.argmax(experience_penalty)
experience_scores = np.linalg.norm(
    experience_scaled - experience_centers[worst_experience_cluster], axis=1
)

engagement_scores_df = pd.DataFrame({
    'MSISDN': engagement_data['MSISDN'],
    'engagement_score': engagement_scores
})

experience_scores_df = pd.DataFrame({
    'MSISDN': experience_data['MSISDN'],
    'experience_score': experience_scores
})

scores_df = engagement_scores_df.merge(
    experience_scores_df, on='MSISDN', how='inner'
)

scores_df['satisfaction_score'] = (scores_df['engagement_score'] + scores_df['experience_score']) / 2
scores_df.head()

,MSISDN,engagement_score,experience_score,satisfaction_score
0,3.360100e+10,0.044497,0.321810,0.183154
1,3.360100e+10,0.057029,0.377816,0.217423
2,3.360100e+10,0.022533,0.092728,0.057630
3,3.360101e+10,0.023165,0.366404,0.194785
4,3.360101e+10,0.124279,0.449604,0.286941


In [33]:
# Combine engagement and experience features for downstream analysis
scores_full = scores_df.merge(
    engagement_data,
    on='MSISDN',
    how='left'
).merge(
    experience_data,
    on='MSISDN',
    how='left',
    suffixes=('_eng', '_exp')
)

# Keep relevant modeling features
model_features = [
    'Session Frequency',
    'Total Session Duration',
    'Total Traffic (Bytes)',
    'Average TCP Retransmission',
    'Average RTT',
    'Average Throughput'
]

scores_full = scores_full[['MSISDN', 'Handset Type', 'engagement_score', 'experience_score', 'satisfaction_score',
                           'Engagement Cluster', 'Experience Cluster'] + model_features]
scores_full.head()

,MSISDN,Handset Type,engagement_score,experience_score,satisfaction_score,Engagement Cluster,Experience Cluster,Session Frequency,Total Session Duration,Total Traffic (Bytes),Average TCP Retransmission,Average RTT,Average Throughput
0,3.360100e+10,Huawei P20 Lite Huawei Nova 3E,0.044497,0.321810,0.183154,1,0,1,116720.0,8.786906e+08,1.078479e+07,23.000000,38.000000
1,3.360100e+10,Apple iPhone 7 (A1778),0.057029,0.377816,0.217423,1,0,1,181230.0,1.568596e+08,1.078479e+07,15.500000,49.500000
2,3.360100e+10,undefined,0.022533,0.092728,0.057630,1,2,1,134969.0,5.959665e+08,1.078479e+07,63.729294,48.500000
3,3.360101e+10,Apple iPhone 5S (A1457),0.023165,0.366404,0.194785,1,2,1,49878.0,4.223207e+08,3.803623e+05,42.000000,124.000000
4,3.360101e+10,Apple iPhone Se (A1723),0.124279,0.449604,0.286941,2,1,2,37104.0,1.457411e+09,7.735101e+06,29.750000,10551.357162


In [34]:
# Task 4.2: Satisfaction Score Reporting
scores_full.sort_values('satisfaction_score', ascending=False, inplace=True)

top_10_satisfied = scores_full[['MSISDN', 'engagement_score', 'experience_score', 'satisfaction_score']].head(10)
print("Top 10 Satisfied Customers:")
print(top_10_satisfied.to_string(index=False))

Top 10 Satisfied Customers:
      MSISDN  engagement_score  experience_score  satisfaction_score
3.361489e+10          1.569979          0.312406            0.941192
3.362578e+10          1.534869          0.347052            0.940961
3.365973e+10          1.509164          0.368176            0.938670
3.362632e+10          1.575372          0.237133            0.906253
3.367588e+10          1.442701          0.330407            0.886554
3.366471e+10          1.171466          0.471348            0.821407
3.365936e+10          1.103507          0.510077            0.806792
3.376054e+10          1.463630          0.090581            0.777105
3.376041e+10          1.157058          0.389975            0.773516
3.366716e+10          1.090935          0.447601            0.769268


In [35]:
# Task 4.3: Regression Model to Predict Satisfaction Score
X = scores_full[model_features]
y = scores_full['satisfaction_score']

# Train/test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg_model = LinearRegression()
reg_model.fit(X_train, y_train)

y_pred = reg_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Linear Regression MSE: {mse:.6f}")

coefficients = pd.Series(reg_model.coef_, index=model_features)
print("\nFeature Coefficients:")
print(coefficients.sort_values(ascending=False))

Linear Regression MSE: 0.001437

Feature Coefficients:
Session Frequency             8.015507e-03
Average Throughput            1.245547e-05
Total Session Duration        1.225635e-07
Total Traffic (Bytes)         1.562985e-11
Average TCP Retransmission   -4.147521e-09
Average RTT                  -1.895743e-03
dtype: float64


In [36]:
# Task 4.4 & 4.5: K-Means (k=2) on Engagement & Experience Scores
score_scaler = StandardScaler()
score_features = score_scaler.fit_transform(scores_full[['engagement_score', 'experience_score']])

score_kmeans = KMeans(n_clusters=2, random_state=42, n_init=10)
scores_full['Satisfaction Cluster'] = score_kmeans.fit_predict(score_features)

cluster_averages = scores_full.groupby('Satisfaction Cluster').agg({
    'experience_score': 'mean',
    'satisfaction_score': 'mean',
    'engagement_score': 'mean',
    'MSISDN': 'count'
}).rename(columns={'MSISDN': 'User Count'})

print("Cluster Averages (experience & satisfaction scores):")
print(cluster_averages[['experience_score', 'satisfaction_score', 'engagement_score', 'User Count']])

Cluster Averages (experience & satisfaction scores):
                      experience_score  satisfaction_score  engagement_score  \
Satisfaction Cluster                                                           
0                             0.348069            0.197686          0.047304   
1                             0.358823            0.304359          0.249894   

                      User Count  
Satisfaction Cluster              
0                          98488  
1                           8368  


In [37]:
# Task 4.6: Export Scores to MySQL and Verify
required_env_vars = ['MYSQL_USER', 'MYSQL_PASSWORD', 'MYSQL_HOST', 'MYSQL_PORT', 'MYSQL_DB']
mysql_config = {var: os.getenv(var) for var in required_env_vars}

missing_vars = [k for k, v in mysql_config.items() if not v]
if missing_vars:
    print("MySQL export skipped: missing environment variables ->", ', '.join(missing_vars))
else:
    mysql_conn_str = (
        f"mysql+pymysql://{mysql_config['MYSQL_USER']}:{mysql_config['MYSQL_PASSWORD']}@"
        f"{mysql_config['MYSQL_HOST']}:{mysql_config['MYSQL_PORT']}/{mysql_config['MYSQL_DB']}"
    )
    try:
        mysql_engine = create_engine(mysql_conn_str)
        export_columns = ['MSISDN', 'engagement_score', 'experience_score', 'satisfaction_score', 'Satisfaction Cluster']
        scores_full[export_columns].to_sql('user_satisfaction_scores', mysql_engine, if_exists='replace', index=False)
        verification_df = pd.read_sql_query('SELECT * FROM user_satisfaction_scores LIMIT 10;', mysql_engine)
        print("Verification sample from MySQL export:")
        display(verification_df)
    except Exception as exc:
        print("MySQL export failed:", exc)
        print("Ensure the database is reachable and credentials are correct before re-running this cell.")

MySQL export skipped: missing environment variables -> MYSQL_USER, MYSQL_PASSWORD, MYSQL_HOST, MYSQL_PORT, MYSQL_DB


## Satisfaction Analysis Summary
- Engagement and experience metrics were recomputed using actual `xdr_data` columns, with missing values and outliers handled via mean imputation and IQR capping.
- Scores were derived as Euclidean distances from the least engaged and worst experience clusters (k=3), and averaged to form the satisfaction score.
- A linear regression model (MSE reported) predicts satisfaction from combined engagement/experience metrics.
- K-means with k=2 segmented users by engagement/experience scores, and per-cluster averages allow targeted follow-up actions.
- Final scores (MSISDN, engagement, experience, satisfaction, cluster) are exported to the `user_satisfaction_scores` table in MySQL, with an inline verification query to confirm persistence.


In [38]:
# Task 4.7: Persist satisfaction scores for dashboard consumption
project_root = Path.cwd().parent
artifact_dir = project_root / "notebooks" / "artifacts"
legacy_path = Path.cwd() / "notebooks" / "artifacts" / "scores_full.parquet"
if legacy_path.exists():
    legacy_path.unlink()
artifact_dir.mkdir(parents=True, exist_ok=True)
artifact_path = artifact_dir / "scores_full.parquet"
scores_full.to_parquet(artifact_path, index=False)
print(f"Saved satisfaction scores to {artifact_path}")

Saved satisfaction scores to d:\Projects\Telcom\notebooks\artifacts\scores_full.parquet
